In [53]:
import pandas as pd
import backtesting
# from backtesting import Backtest, Strategy
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import *
import numpy as np
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

import ipywidgets as widgets

In [54]:
data = pd.read_csv('data/decisions_EURUSD_5Y.csv')

In [55]:
data.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,Name,Return,Up,...,MACD_decision,diff_MACD,diff_MACD_prev,fuzz_MACD_strategy,dec_bollinger,diff_cl_BU,diff_cl_BL,prev_diff_cl_BU,prev_diff_cl_BL,fuzz_BB_strategy
0,2019-05-02,1.120185,1.121818,1.117356,1.120084,1.120084,0,EURUSD=X,-0.001356,0.000000,...,0,0.000048,0.000000,0,0,-0.013554,0.006675,-0.012144,0.007844,0
1,2019-05-03,1.117256,1.120072,1.114132,1.117356,1.117356,0,EURUSD=X,-0.002439,0.000000,...,-1,-0.000081,0.000048,0,0,-0.016387,0.004574,-0.013554,0.006675,0
2,2019-05-06,1.118706,1.120448,1.117993,1.118581,1.118581,0,EURUSD=X,0.001096,0.001096,...,0,-0.000063,-0.000081,0,0,-0.015189,0.006163,-0.016387,0.004574,0
3,2019-05-07,1.120448,1.121957,1.117606,1.120185,1.120185,0,EURUSD=X,0.001433,0.001433,...,1,0.000072,-0.000063,0,0,-0.013316,0.008045,-0.015189,0.006163,0
4,2019-05-08,1.118956,1.121500,1.118956,1.119107,1.119107,0,EURUSD=X,-0.000963,0.000000,...,0,0.000100,0.000072,0,0,-0.013974,0.007306,-0.013316,0.008045,0
5,2019-05-09,1.119746,1.125100,1.117568,1.119833,1.119833,0,EURUSD=X,0.000649,0.000649,...,0,0.000176,0.000100,0,0,-0.012623,0.008189,-0.013974,0.007306,0
6,2019-05-10,1.121856,1.125239,1.121856,1.122208,1.122208,0,EURUSD=X,0.002119,0.002119,...,0,0.000382,0.000176,0,0,-0.009882,0.010585,-0.012623,0.008189,0
7,2019-05-13,1.123671,1.126342,1.122410,1.123646,1.123646,0,EURUSD=X,0.001280,0.001280,...,0,0.000599,0.000382,0,0,-0.007310,0.011574,-0.009882,0.010585,0
8,2019-05-14,1.123242,1.124606,1.120486,1.123255,1.123255,0,EURUSD=X,-0.000348,0.000000,...,0,0.000692,0.000599,0,0,-0.006398,0.010604,-0.007310,0.011574,0
9,2019-05-15,1.120360,1.122272,1.118043,1.120448,1.120448,0,EURUSD=X,-0.002502,0.000000,...,0,0.000548,0.000692,0,0,-0.008067,0.007465,-0.006398,0.010604,0


In [56]:
data.columns.tolist()

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Adj Close',
 'Volume',
 'Name',
 'Return',
 'Up',
 'Down',
 'avg_14up',
 'avg_14down',
 'RS_14',
 'RSI',
 '5MA',
 '15MA',
 '12Ewm',
 '26Ewm',
 'MACD',
 'low_14',
 'high_14',
 'k_percent',
 'r_percent',
 'B_MA',
 'BU',
 'BL',
 'dec_rsi',
 'Signal_RSI_fuzz_system',
 'dec_will',
 'Signal_william_fuzz_system',
 'prev_5MA',
 'crossover',
 'cross_MA',
 'diff_MA',
 'diff_MA_prev',
 'fuzz_MA_strategy',
 'Signal_Line',
 'MACD_decision',
 'diff_MACD',
 'diff_MACD_prev',
 'fuzz_MACD_strategy',
 'dec_bollinger',
 'diff_cl_BU',
 'diff_cl_BL',
 'prev_diff_cl_BU',
 'prev_diff_cl_BL',
 'fuzz_BB_strategy']

### Sprawdzanie sygnałów w kolumnach ostrych i rozmytych

In [57]:
# Zliczenie przypadków, w których wartości są przeciwne

rsi_count = ((data['dec_rsi'] == 1) & (data['Signal_RSI_fuzz_system'] == -1) | (data['dec_rsi'] == -1) & (data['Signal_RSI_fuzz_system'] == 1)).sum()
print("rsi_count", rsi_count)

williams_count = ((data['dec_will'] == 1) & (data['Signal_william_fuzz_system'] == -1) | (data['dec_will'] == -1) & (data['Signal_william_fuzz_system'] == 1)).sum()
print("williams_count", williams_count)

# cross_MA - ostre decyzje
ma_count = ((data['cross_MA'] == 1) & (data['fuzz_MA_strategy'] == -1) | (data['cross_MA'] == -1) & (data['fuzz_MA_strategy'] == 1)).sum()
print("ma_count", ma_count)

macd_count = ((data['MACD_decision'] == 1) & (data['fuzz_MACD_strategy'] == -1) | (data['MACD_decision'] == -1) & (data['fuzz_MACD_strategy'] == 1)).sum()
print("macd_count", macd_count)


bb_count = ((data['dec_bollinger'] == 1) & (data['fuzz_BB_strategy'] == -1) | (data['dec_bollinger'] == -1) & (data['fuzz_BB_strategy'] == 1)).sum()

print("bb_count", bb_count)


rsi_count 0
williams_count 0
ma_count 1
macd_count 0
bb_count 0


In [58]:
opposite_rows = data[((data['dec_rsi'] == 1) & (data['Signal_RSI_fuzz_system'] == -1)) | ((data['dec_rsi'] == -1) & (data['Signal_RSI_fuzz_system'] == 1))]
opposite_rows[['Date','dec_rsi', 'Signal_RSI_fuzz_system', 'RS_14', 'RSI']]


,Date,dec_rsi,Signal_RSI_fuzz_system,RS_14,RSI


In [59]:
opposite_rows = data[((data['cross_MA'] == 1) & (data['fuzz_MA_strategy'] == -1)) | ((data['cross_MA'] == -1) & (data['fuzz_MA_strategy'] == 1))]
opposite_rows[['Date', '5MA', '15MA', 'cross_MA', 'fuzz_MA_strategy', 'diff_MA', 'diff_MA_prev']]


,Date,5MA,15MA,cross_MA,fuzz_MA_strategy,diff_MA,diff_MA_prev
251,2020-04-20,1.090778,1.090844,-1,1,-0.000066,-0.000566


<img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExNWcyanR3eWUxYTRsNHpyZTAxbmoyNHcxOTJnbWtqcnl5NHRyN2hqMCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/jPAdK8Nfzzwt2/giphy.gif" width="350" align="center">

Na ten moment nie wiem czemu tutaj jest rozbieżność

In [60]:
data = data.set_index(pd.DatetimeIndex(data['Date']))

## Tylko sygnały

### Ostre

In [61]:
class CustomStrategy(SignalStrategy):
    def init(self):
        super().init()
        
        self.set_signal(data['dec_rsi'])
      
        
bt = Backtest(data, CustomStrategy, cash=10000, 
              commission=0.002, trade_on_close=False, hedging = False, exclusive_orders = True)

stats = bt.run()
bt.plot()
stats

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   96.028037
Equity Final [$]                   9999.21572
Equity Peak [$]                       10000.0
Return [%]                          -0.007843
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                   -0.001539
Volatility (Ann.) [%]                  0.0008
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.007968
Avg. Drawdown [%]                   -0.007968
Max. Drawdown Duration     1727 days 00:00:00
Avg. Drawdown Duration     1727 days 00:00:00
# Trades                                  304
Win Rate [%]                        34.868421
Best Trade [%]                       3.345059
Worst Trade [%]                     -4.160291
Avg. Trade [%]                    

### Rozmyte

In [62]:
class CustomStrategy(SignalStrategy):
    def init(self):
        super().init()
        
        self.set_signal(data['Signal_RSI_fuzz_system'])
      
        
bt = Backtest(data, CustomStrategy, cash=10000, 
              commission=0.002, trade_on_close=False, hedging = False, exclusive_orders = True)

stats = bt.run()
bt.plot()
stats

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   97.819315
Equity Final [$]                  9999.212488
Equity Peak [$]                  10000.008594
Return [%]                          -0.007875
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                   -0.001546
Volatility (Ann.) [%]                0.000806
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.008065
Avg. Drawdown [%]                   -0.002698
Max. Drawdown Duration     1750 days 00:00:00
Avg. Drawdown Duration      585 days 00:00:00
# Trades                                  386
Win Rate [%]                        35.751295
Best Trade [%]                       2.784302
Worst Trade [%]                     -4.588674
Avg. Trade [%]                    

## Strategia z tp i sl

### Ostre

In [63]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['dec_rsi'], 'rsi')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
           ssl=[0.01, 0.1],
           size_tr=[0.4, 1], 
           maximize='Equity Final [$]',
           method='skopt',
           max_tries=1000,
           random_state=10,
           return_heatmap=True,
           return_optimization=True)
heatmap.sort_values().iloc[-3:]

Backtest.optimize:   0%|          | 0/1000 [00:00<?, ?it/s]

ttp       ssl       size_tr 
0.054764  0.095126  0.989563    11091.973290
0.057007  0.099007  0.997161    11132.834576
0.052925  0.094504  0.994672    11135.435991
Name: Equity Final [$], dtype: float64

In [64]:
stats_skopt

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   97.663551
Equity Final [$]                 11135.435991
Equity Peak [$]                  11460.742845
Return [%]                           11.35436
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                    2.133176
Volatility (Ann.) [%]                5.166405
Sharpe Ratio                         0.412894
Sortino Ratio                         0.63225
Calmar Ratio                         0.307053
Max. Drawdown [%]                   -6.947265
Avg. Drawdown [%]                   -1.141979
Max. Drawdown Duration      453 days 00:00:00
Avg. Drawdown Duration       53 days 00:00:00
# Trades                                   64
Win Rate [%]                            68.75
Best Trade [%]                       5.253578
Worst Trade [%]                     -9.319809
Avg. Trade [%]                    

### Rozmyte

In [65]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['Signal_RSI_fuzz_system'], 'rsi')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
           ssl=[0.01, 0.1],
           size_tr=[0.4, 1], 
           maximize='Equity Final [$]',
           method='skopt',
           max_tries=1000,
           random_state=10,
           return_heatmap=True,
           return_optimization=True)
heatmap.sort_values().iloc[-3:]

Backtest.optimize:   0%|          | 0/1000 [00:00<?, ?it/s]

ttp       ssl       size_tr 
0.030858  0.013006  0.998991    11808.270130
0.030658  0.012796  0.998083    11838.864439
0.030685  0.012820  0.999651    11952.694992
Name: Equity Final [$], dtype: float64

In [66]:
stats_skopt

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   86.682243
Equity Final [$]                 11952.694992
Equity Peak [$]                  12013.917435
Return [%]                           19.52695
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                     3.56275
Volatility (Ann.) [%]                6.490943
Sharpe Ratio                          0.54888
Sortino Ratio                        0.843049
Calmar Ratio                         0.363967
Max. Drawdown [%]                    -9.78865
Avg. Drawdown [%]                   -1.891907
Max. Drawdown Duration      667 days 00:00:00
Avg. Drawdown Duration       60 days 00:00:00
# Trades                                  151
Win Rate [%]                        73.509934
Best Trade [%]                       3.749728
Worst Trade [%]                     -2.658748
Avg. Trade [%]                    

***
***

## Williams %R

### Ostre

In [67]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['dec_will'], 'williams')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
           ssl=[0.01, 0.1],
           size_tr=[0.4, 1], 
           maximize='Equity Final [$]',
           method='skopt',
           max_tries=1000,
           random_state=10,
           return_heatmap=True,
           return_optimization=True)
heatmap.sort_values().iloc[-3:]

Backtest.optimize:   0%|          | 0/1000 [00:00<?, ?it/s]

ttp       ssl       size_tr 
0.058337  0.025705  0.996042    11974.215649
0.058093  0.023688  0.996118    12032.348821
0.056722  0.023124  0.992232    12231.850239
Name: Equity Final [$], dtype: float64

In [68]:
stats_skopt

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   98.364486
Equity Final [$]                 12231.850239
Equity Peak [$]                  12300.689358
Return [%]                          22.318502
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                    4.033056
Volatility (Ann.) [%]                6.310942
Sharpe Ratio                         0.639058
Sortino Ratio                        1.024036
Calmar Ratio                         0.515871
Max. Drawdown [%]                    -7.81795
Avg. Drawdown [%]                   -1.354844
Max. Drawdown Duration      603 days 00:00:00
Avg. Drawdown Duration       65 days 00:00:00
# Trades                                  172
Win Rate [%]                        73.255814
Best Trade [%]                       6.186862
Worst Trade [%]                     -2.978993
Avg. Trade [%]                    

### Rozmyte

In [69]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['Signal_william_fuzz_system'], 'williams')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
           ssl=[0.01, 0.1],
           size_tr=[0.4, 1], 
           maximize='Equity Final [$]',
           method='skopt',
           max_tries=1000,
           random_state=10,
           return_heatmap=True,
           return_optimization=True)
heatmap.sort_values().iloc[-3:]

Backtest.optimize:   0%|          | 0/1000 [00:00<?, ?it/s]

ttp       ssl       size_tr 
0.044181  0.037050  0.854939    10906.739816
0.055807  0.036894  0.854902    11051.874310
0.068422  0.036482  0.855100    11292.482516
Name: Equity Final [$], dtype: float64

In [70]:
stats_skopt

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   99.299065
Equity Final [$]                 11292.482516
Equity Peak [$]                  11359.561281
Return [%]                          12.924825
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                    2.414286
Volatility (Ann.) [%]                4.115789
Sharpe Ratio                         0.586591
Sortino Ratio                        0.969419
Calmar Ratio                         0.409914
Max. Drawdown [%]                    -5.88974
Avg. Drawdown [%]                   -1.067492
Max. Drawdown Duration      710 days 00:00:00
Avg. Drawdown Duration       74 days 00:00:00
# Trades                                  138
Win Rate [%]                        63.768116
Best Trade [%]                       6.040765
Worst Trade [%]                      -5.21014
Avg. Trade [%]                    

## MA

### Ostre

In [71]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['cross_MA'], 'ma')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
           ssl=[0.01, 0.1],
           size_tr=[0.4, 1], 
           maximize='Equity Final [$]',
           method='skopt',
           max_tries=1000,
           random_state=10,
           return_heatmap=True,
           return_optimization=True)
heatmap.sort_values().iloc[-3:]

Backtest.optimize:   0%|          | 0/1000 [00:00<?, ?it/s]

ttp       ssl       size_tr 
0.044757  0.020535  0.999667    12501.231202
0.045057  0.020485  0.986704    12558.135455
0.045289  0.019810  0.958761    12633.246016
Name: Equity Final [$], dtype: float64

In [72]:
stats_skopt

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   84.501558
Equity Final [$]                 12633.246016
Equity Peak [$]                  12702.162577
Return [%]                           26.33246
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                     4.69441
Volatility (Ann.) [%]                5.810547
Sharpe Ratio                         0.807912
Sortino Ratio                        1.372527
Calmar Ratio                         0.839561
Max. Drawdown [%]                   -5.591507
Avg. Drawdown [%]                   -0.987258
Max. Drawdown Duration      258 days 00:00:00
Avg. Drawdown Duration       30 days 00:00:00
# Trades                                   60
Win Rate [%]                             45.0
Best Trade [%]                       5.836652
Worst Trade [%]                     -3.204367
Avg. Trade [%]                    

### Rozmyte

In [73]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['fuzz_MA_strategy'], 'ma')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
           ssl=[0.01, 0.1],
           size_tr=[0.4, 1], 
           maximize='Equity Final [$]',
           method='skopt',
           max_tries=1000,
           random_state=10,
           return_heatmap=True,
           return_optimization=True)
heatmap.sort_values().iloc[-3:]

Backtest.optimize:   0%|          | 0/1000 [00:00<?, ?it/s]

ttp       ssl       size_tr 
0.084787  0.076009  0.999295    11947.911795
0.085292  0.076027  0.999263    11962.704250
0.085312  0.076365  0.999898    11967.035173
Name: Equity Final [$], dtype: float64

In [74]:
stats_skopt

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   87.538941
Equity Final [$]                 11967.035173
Equity Peak [$]                  12290.993216
Return [%]                          19.670352
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                    3.587124
Volatility (Ann.) [%]                6.625045
Sharpe Ratio                         0.541449
Sortino Ratio                        0.831937
Calmar Ratio                         0.365088
Max. Drawdown [%]                   -9.825373
Avg. Drawdown [%]                   -1.679408
Max. Drawdown Duration      717 days 00:00:00
Avg. Drawdown Duration       44 days 00:00:00
# Trades                                   21
Win Rate [%]                        66.666667
Best Trade [%]                        8.53599
Worst Trade [%]                     -7.812477
Avg. Trade [%]                    

## MACD

### Ostre

In [75]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['MACD_decision'], 'macd')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
           ssl=[0.01, 0.1],
           size_tr=[0.4, 1], 
           maximize='Equity Final [$]',
           method='skopt',
           max_tries=1000,
           random_state=10,
           return_heatmap=True,
           return_optimization=True)
heatmap.sort_values().iloc[-3:]

Backtest.optimize:   0%|          | 0/1000 [00:00<?, ?it/s]

ttp       ssl       size_tr 
0.053385  0.015904  0.942133    13083.301282
0.053933  0.016725  0.940332    13137.367972
0.053916  0.017075  0.943656    13303.599798
Name: Equity Final [$], dtype: float64

In [76]:
stats_skopt

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   91.277259
Equity Final [$]                 13303.599798
Equity Peak [$]                  13387.047964
Return [%]                          33.035998
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                    5.762181
Volatility (Ann.) [%]                5.726786
Sharpe Ratio                         1.006181
Sortino Ratio                        1.717495
Calmar Ratio                         0.921583
Max. Drawdown [%]                   -6.252485
Avg. Drawdown [%]                   -1.013905
Max. Drawdown Duration      294 days 00:00:00
Avg. Drawdown Duration       29 days 00:00:00
# Trades                                   71
Win Rate [%]                         60.56338
Best Trade [%]                       5.145855
Worst Trade [%]                     -2.271293
Avg. Trade [%]                    

In [77]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['fuzz_MACD_strategy'], 'macd')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
           ssl=[0.01, 0.1],
           size_tr=[0.4, 1], 
           maximize='Equity Final [$]',
           method='skopt',
           max_tries=1000,
           random_state=10,
           return_heatmap=True,
           return_optimization=True)
heatmap.sort_values().iloc[-3:]

Backtest.optimize:   0%|          | 0/1000 [00:00<?, ?it/s]

ttp       ssl       size_tr 
0.047135  0.071098  0.965441    12990.122138
0.046974  0.068273  0.964821    12995.422442
0.046979  0.068388  0.965833    13027.479228
Name: Equity Final [$], dtype: float64

In [78]:
stats_skopt

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   79.906542
Equity Final [$]                 13027.479228
Equity Peak [$]                  13096.458447
Return [%]                          30.274792
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                    5.327722
Volatility (Ann.) [%]                5.848334
Sharpe Ratio                         0.910981
Sortino Ratio                        1.474316
Calmar Ratio                         0.650538
Max. Drawdown [%]                   -8.189724
Avg. Drawdown [%]                   -1.080045
Max. Drawdown Duration      644 days 00:00:00
Avg. Drawdown Duration       29 days 00:00:00
# Trades                                   26
Win Rate [%]                        76.923077
Best Trade [%]                       4.987095
Worst Trade [%]                     -6.976193
Avg. Trade [%]                    

## Bollinger Bands 

### Ostre

In [79]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['dec_bollinger'], 'bollinger')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
           ssl=[0.01, 0.1],
           size_tr=[0.4, 1], 
           maximize='Equity Final [$]',
           method='skopt',
           max_tries=1000,
           random_state=10,
           return_heatmap=True,
           return_optimization=True)
heatmap.sort_values().iloc[-3:]

Backtest.optimize:   0%|          | 0/1000 [00:00<?, ?it/s]

ttp       ssl       size_tr 
0.098619  0.061252  0.999866    12727.525823
0.095095  0.073534  0.999983    12937.786426
0.096254  0.061343  0.997472    13020.341810
Name: Equity Final [$], dtype: float64

In [80]:
stats_skopt

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   95.404984
Equity Final [$]                  13020.34181
Equity Peak [$]                  13131.402563
Return [%]                          30.203418
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                    5.316394
Volatility (Ann.) [%]                5.301298
Sharpe Ratio                         1.002848
Sortino Ratio                        1.646532
Calmar Ratio                         1.025678
Max. Drawdown [%]                   -5.183298
Avg. Drawdown [%]                   -0.876097
Max. Drawdown Duration      478 days 00:00:00
Avg. Drawdown Duration       32 days 00:00:00
# Trades                                   64
Win Rate [%]                            81.25
Best Trade [%]                       9.382553
Worst Trade [%]                     -5.894845
Avg. Trade [%]                    

### Rozmyte

In [81]:
class CustomStrategy(SignalStrategy):
    
    ssl = 0.046
    ttp = 0.054
    size_tr = 0.9
    def init(self):
        
        super().init()
        self.rsi = self.I(lambda x: data['fuzz_BB_strategy'], 'bollinger')

    def next(self):
        
        close = self.data.Close
        signal = self.rsi
        #prev_signal = self.rsi[-1]
        
        upper_tp = close*(1+self.ttp) 
        upper_sl = close*(1+self.ssl) 
        lower_sl = close*(1-self.ssl)   
        lower_tp = close*(1-self.ttp)
        
        #size_transaction = 0.85
        
        #if not signal == prev_signal:
        if signal == 1 and not self.position.is_long:
            self.buy(size=self.size_tr, sl=lower_sl, tp=upper_tp)
        elif signal == -1 and not self.position.is_short:
            self.sell(size=self.size_tr, sl = upper_sl, tp=lower_tp)
                
bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False)
stats = bt.run()
bt.plot()
stats

#optymalizacja parametrow
stats_skopt, heatmap, optimize_result = bt.optimize(ttp=[0.01, 0.1],
           ssl=[0.01, 0.1],
           size_tr=[0.4, 1], 
           maximize='Equity Final [$]',
           method='skopt',
           max_tries=1000,
           random_state=10,
           return_heatmap=True,
           return_optimization=True)
heatmap.sort_values().iloc[-3:]

Backtest.optimize:   0%|          | 0/1000 [00:00<?, ?it/s]

ttp       ssl       size_tr 
0.088490  0.092539  0.999098    13724.011145
0.087161  0.075257  0.999076    13746.181324
0.063006  0.099444  0.999104    14109.247258
Name: Equity Final [$], dtype: float64

In [82]:
stats_skopt

Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   97.196262
Equity Final [$]                 14109.247258
Equity Peak [$]                   14494.58194
Return [%]                          41.092473
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                    6.989677
Volatility (Ann.) [%]                6.228576
Sharpe Ratio                         1.122195
Sortino Ratio                        1.850864
Calmar Ratio                         1.002567
Max. Drawdown [%]                   -6.971779
Avg. Drawdown [%]                   -0.835278
Max. Drawdown Duration      203 days 00:00:00
Avg. Drawdown Duration       23 days 00:00:00
# Trades                                   33
Win Rate [%]                        90.909091
Best Trade [%]                       7.340572
Worst Trade [%]                      -3.34573
Avg. Trade [%]                    

## Pozostałe próby 

In [83]:
class CustomStrategy(SignalStrategy):
    
    price_upper = 0.1  # różnica w górę
    price_lower = 0.02  # różnica w dół
    
    def init(self):
        super().init()
        self.rsi = self.I(lambda x: data['dec_rsi'], 'rsi')
        self.last_trade_date = None

    def next(self):
        close = self.data.Close[-1]
        signal = self.rsi[-1] 

        upper = close * (1 + self.price_upper)  # upper jako price_upper% różnica w górę
        lower = close * (1 - self.price_lower)  # lower jako price_lower% różnica w dół   
        
        size_transaction = .7
        
        if signal == -1:
            if self.position.is_long:
                self.sell(size=size_transaction, sl = upper, tp=lower)
            elif self.position.is_short:
                pass
            else:
                self.sell(size=size_transaction, sl = upper, tp=lower)
        elif signal == 1:
            if self.position.is_long:
                pass
            elif self.position.is_short:
                self.buy(size=size_transaction, sl=lower, tp = upper)

            else:
                self.buy(size=size_transaction,sl=lower, tp = upper)



bt = Backtest(data, CustomStrategy, cash=10000, commission=0.002, 
              trade_on_close=False, hedging=False, exclusive_orders=True)
stats = bt.run()
bt.plot()
stats


Start                     2019-05-02 00:00:00
End                       2024-04-03 00:00:00
Duration                   1798 days 00:00:00
Exposure Time [%]                   92.834891
Equity Final [$]                  9327.414247
Equity Peak [$]                  10449.166947
Return [%]                          -6.725858
Buy & Hold Return [%]               -3.859441
Return (Ann.) [%]                   -1.357222
Volatility (Ann.) [%]                4.893971
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -12.201869
Avg. Drawdown [%]                   -2.745491
Max. Drawdown Duration     1465 days 00:00:00
Avg. Drawdown Duration      250 days 00:00:00
# Trades                                   43
Win Rate [%]                        53.488372
Best Trade [%]                       3.131685
Worst Trade [%]                     -9.868757
Avg. Trade [%]                    